# Dependences

In [25]:
include("../../src/struct_data.jl")

save_data

# Model

In [26]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_4_MiniTest",
        path_output = ""
    ) 
)

dump(model)

  0.000002 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 100000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_4_MiniTest"
    path_output: String ""


In [27]:
# Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.06)
Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.12)
agg_size = 15

15

# Aggregate

In [28]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.12)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), AggSimulation(AggParameter(Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 

In [29]:
# # Run fusion of two aggregates
# agg = nothing
# agg = FusionAggregate(
#     [AggType(
#         "HEK_1", 
#         InteractionPar(Par1, Par2),
#         Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
#     )], 
#     model
# )

# kNN

In [30]:
include("../../src/neighbor.jl")

dist = zeros(size(agg.Simulation.Neighbor.idx_red)) |> cu
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

display(agg.Simulation.Neighbor.idx_red)

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  2461  2462  2463  2468  2469  2470
  6   7   8  49   1   2   3   3  58     2502  2503  2504  2498  2499  2500
  5   6   7  50  50   1   2  53  15     2503  2504  2455  2499  2500  2501
 42  43  44   5  51  51  52  54  10     2454  2455  2456  2461  2462  2463
 43  44  45  10   6  52  53   7  49     2453  2454  2500  2460  2461  2462
  2   3   2   9   4   7   8  14   4  …  2498  2499  2495  2503  2502  2503
 35   1  37  40  11   5   6  13  66     2500  2501  2496  2469  2504  2471
 37  36  39  42  10  12  13  44  50     2493  2494  2462  2467  2468  2469
 51  38  53  58  41  11  12  46  48     2494  2495  2464  2453  2470  2455
  0  52   0   0  43  42  43  62   0     2460  2461  2447     0  2454     0
  0   0   0   0  59  44  45   0   0  …  2462  2463     0     0     0     0
  0   0   0   0   0  60  61   0   0     2445  2446     0     0     0     0
  0   0   0   0   0   0   0   0   0        0    

# Sum Forces

In [31]:
include("../../src/forces/forces.jl")

threads=(64,3)
CUDA.@time @cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Force.N_i,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        pi/4,
        model.Time.dt
    )
)

display(agg.Simulation.Force.F)

  0.627085 seconds (355.81 k CPU allocations: 21.010 MiB)


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  7.13207f-12  -1.49224f-5    0.000364654
  7.13207f-12  -1.49224f-5    0.000364654
  0.00465229   -0.110124      0.213565
 -4.6549f-12    0.000133741   0.000364654
  0.0865809     0.0584852    -0.0587886
 -4.6549f-12    0.000164602   0.000364654
 -4.6549f-12    0.000164602   0.000364654
 -0.112405     -0.032632     -0.0258759
  0.043444      0.111392     -0.00900595
  7.43841f-12  -0.000158756   0.000364828
  7.43841f-12  -0.000158756   0.000364828
 -0.0172607     0.0287521     0.115544
  0.0675495    -0.0900011     0.0423819
  ⋮                          
 -7.43841f-12   0.000158756  -0.000364828
 -7.43841f-12   0.000158756  -0.000364828
 -0.0333073    -0.110201      0.0329732
 -3.38513f-5    9.98547f-5   -0.000364828
 -0.0135012    -0.116776      0.024379
  0.0983187     0.0446582     0.0518272
 -0.0952958    -0.0694891    -0.0230104
 -4.6549f-12   -0.000164602  -0.000364654
  8.91939f-6   -0.000149172  -0.000364654
 -7.13207f-12   1

# MiniTest

In [32]:
include("../../src/run_event.jl")

run_test(agg, model, "mini Testing")
display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

mini Testing... 100%|████████████████████████████████████| Time: 0:02:33


1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.00346653    0.00271958   -0.013833
  0.00270668   -0.00729975   -0.0157528
  0.0013952     0.00358732   -0.015256
 -0.00468904   -0.00275245    0.0043112
  0.13163      -0.0231641    -0.0181851
  0.00320982   -0.0046614    -0.0188008
  0.199927      0.0787721    -0.108054
 -0.113613      0.0413568    -0.0551883
 -0.0385023     0.222468      0.0796731
  0.0389753     0.238055     -0.0236828
 -0.0926488     0.0313485    -0.0958241
  0.00606135    0.0069624    -0.0182373
  0.00480723   -0.00288136   -0.00792073
  ⋮                          
  0.0186607     0.113381      0.0309219
  0.000769117   0.00146554    0.0168214
  0.0160925     0.10671      -0.0384187
  0.00502785   -0.0114827     0.015602
 -0.00513367    0.0195587    -0.00628613
  0.0143612    -0.122117      0.204068
  0.0109652     0.00971699    0.005865
  0.00947759    0.000612633   0.00527012
 -0.00831498   -0.00340752   -8.93387f-5
  0.00100021    0.00416696    0.0138451
 

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -3.45564    -0.708483  -12.6441
  0.0730225  -3.71005   -12.6065
  3.84169    -2.42657   -10.5967
 -4.78849    -3.31037   -11.5569
 -1.89446    -3.51073   -12.3627
 -0.786246   -2.36406   -12.8012
  2.29218    -2.58017   -11.0566
  2.87928    -4.04789   -11.1488
 -5.22653    -0.26303   -12.6813
 -2.5699     -2.00025   -12.7315
 -1.53895    -0.74907   -12.7743
  0.394198   -1.23276   -12.6088
  2.12538    -1.45987   -12.4202
  ⋮                     
 -1.06103     0.838748   10.4714
  0.260457    1.12434    11.2455
  0.852867   -0.579585   11.1876
  2.47756    -1.48317    11.0559
 -5.00681     0.897559    8.2358
 -2.1821      3.25115    10.3984
 -1.36927     2.04124     9.42553
  1.54922     0.852859   10.2458
  4.96015     3.1585      7.45184
 -3.97551     1.2735      9.55231
  1.67707     2.21512    10.7978
 -0.12338     2.87608     8.57301